# Importing Libraries and loading dataset

In [3]:
#import all the libraries
import pandas as pd
import datetime as dt


In [4]:
#load datasets into each variable
customers= pd.read_csv(r"C:\olist_customers_dataset.csv")
geo_location=pd.read_csv(r"C:\olist_geolocation_dataset.csv")
products=pd.read_csv(r"C:\olist_products_dataset.csv")
orders=pd.read_csv(r"C:\olist_orders_dataset.csv")
order_reviews=pd.read_csv(r"C:\olist_order_reviews_dataset_translated - olist_order_reviews_dataset.csv.csv")
order_items=pd.read_csv(r"C:\olist_order_items_dataset.csv")
order_payments=pd.read_csv(r"C:\olist_order_payments_dataset.csv")
product_translation=pd.read_csv(r"C:\product_category_name_translation.csv")


# Understanding the Dataset (EDA and Data Preprocessing each dataset for merging)

In [5]:
#check dataset info
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [6]:
#convert time columns to datetime format
time_columns=['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date','order_delivered_customer_date',
               'order_estimated_delivery_date']
for col in time_columns:
    orders[col]=pd.to_datetime(orders[col])

## Adiitional columns are added in the following and data is transformed

In [7]:
#calculate delivery timeframe
orders['delivery_timeframe']=orders['order_estimated_delivery_date']-orders['order_delivered_customer_date']
orders['delivery_timeframe (days)']=orders['delivery_timeframe'].dt.days
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_timeframe,delivery_timeframe (days)
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7 days 02:34:47,7.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,5 days 08:32:15,5.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,17 days 05:53:31,17.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,12 days 23:31:18,12.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,9 days 05:42:58,9.0


In [8]:
#drop unused columns
orders.drop(columns=['delivery_timeframe'],inplace=True)

In [9]:
#drop unused columns
orders.drop(columns=['order_approved_at','order_delivered_carrier_date','order_estimated_delivery_date'],inplace=True)

In [10]:
#fill nan values with unknown
orders['order_delivered_customer_date']=orders['order_delivered_customer_date'].fillna('unknown')
orders['delivery_timeframe (days)']=orders['delivery_timeframe (days)'].fillna('unknown')

orders.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_delivered_customer_date    0
delivery_timeframe (days)        0
dtype: int64

In [11]:
#final check of dataset
order_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   review_id                       99224 non-null  object
 1   order_id                        99224 non-null  object
 2   review_score                    99224 non-null  int64 
 3   review_comment_title            11566 non-null  object
 4   review_comment_message          40969 non-null  object
 5   review_comment_message_english  40967 non-null  object
 6   review_creation_date            99224 non-null  object
 7   review_answer_timestamp         99224 non-null  object
dtypes: int64(1), object(7)
memory usage: 6.1+ MB


Added a classification column to order_reviews based on review scores, categorizing them as Positive, Negative, or Neutral, based on the rating and dropped unnecessary columns.

In [12]:
#add review score classification column
def classify_review_scores(x):
    if x>=4:
        return 'Positive'
    elif x<=2:
        return 'Negative'
    else:
        return 'Neutral'

order_reviews['review_score_classification']=order_reviews['review_score'].apply(classify_review_scores)


columns=['review_comment_title','review_creation_date','review_answer_timestamp','review_comment_message']
order_reviews.drop(columns=columns,inplace=True)

order_reviews.head(10)

,review_id,order_id,review_score,review_comment_message_english,review_score_classification
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,Positive
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,Positive
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,Positive
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,I received it well before the stipulated deadl...,Positive
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,"Congratulations lannister stores, I loved shop...",Positive
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,NaN,Negative
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,NaN,Positive
7,7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,NaN,Positive
8,a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,NaN,Positive
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,efficient device. On the website the brand of ...,Positive


In [13]:
#remove any duplicates and fill NaN values
order_reviews.duplicated().sum()
order_reviews.fillna('Unknown',inplace=True)

In [14]:
#check for null values
order_reviews.isnull().sum()

review_id                         0
order_id                          0
review_score                      0
review_comment_message_english    0
review_score_classification       0
dtype: int64

In [15]:
#check dataset info and keep only relevant columns
order_payments.info()
order_payments=order_payments[['order_id','payment_type']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [16]:
#check dataset info and keep only relevant columns
order_items.info()
order_items=order_items[['order_id','order_item_id','product_id','price']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [17]:
#check dataset info and keep only relevant columns
products.info()
product_translation.info()

products=products[['product_id','product_category_name']]
products_merged=pd.merge(products,product_translation,on='product_category_name')
products_merged.drop(['product_category_name'],axis=1,inplace=True)
products_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  

,product_id,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery


The product dataset is reviewed to identify any inconsistencies or missing values in the category column

In [18]:
#get unique values in column 'product_category_name_english'
products_merged.product_category_name_english.unique()

array(['perfumery', 'art', 'sports_leisure', 'baby', 'housewares',
       'musical_instruments', 'cool_stuff', 'furniture_decor',
       'home_appliances', 'toys', 'bed_bath_table',
       'construction_tools_safety', 'computers_accessories',
       'health_beauty', 'luggage_accessories', 'garden_tools',
       'office_furniture', 'auto', 'electronics', 'fashion_shoes',
       'telephony', 'stationery', 'fashion_bags_accessories', 'computers',
       'home_construction', 'watches_gifts',
       'construction_tools_construction', 'pet_shop', 'small_appliances',
       'agro_industry_and_commerce', 'furniture_living_room',
       'signaling_and_security', 'air_conditioning', 'consoles_games',
       'books_general_interest', 'costruction_tools_tools',
       'fashion_underwear_beach', 'fashion_male_clothing',
       'kitchen_dining_laundry_garden_furniture',
       'industry_commerce_and_business', 'fixed_telephony',
       'construction_tools_lights', 'books_technical',
       'home_app

Some category names appear to be inaccurate or too similar, such as 'home_comfort' and 'home_comfort2,' as well as different variations of 'home_appliances.' To resolve this issue, these values will be standardized and corrected.

In [19]:
#create dictionary for replacing values
mapping = {
    "home_confort": "home_comfort",
    "home_comfort_2": "home_comfort",
    "home_appliances_2": "home_appliances"
}

#mapping to new values and fill NaN with 'unknown'
products_merged["product_category_name_english"] = products_merged["product_category_name_english"].replace(mapping)
products_merged["product_category_name_english"] = products_merged["product_category_name_english"].fillna('unknown')
products_merged

,product_id,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery
...,...,...
32323,13b25797ad1e6d6c7cd3cbeb35f8ee7a,furniture_mattress_and_upholstery
32324,dc36a7859b743d8610a2bbbaea26ece9,furniture_mattress_and_upholstery
32325,107fde0930956120d1e13dd1062fbb46,furniture_mattress_and_upholstery
32326,726b4e18f00255e2e63491bcba3f60b8,furniture_mattress_and_upholstery


In [20]:
#check datasets info
customers.info()
geo_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  

In [21]:
#check for duplicated values
geo_location.duplicated().sum()
geo_location.drop_duplicates(inplace=True)
geo_location.shape

(738332, 5)

In [22]:
customers.duplicated().sum()

0

The final step is to merge all the cleaned datasets. To ensure faster and more efficient loading, we will create two separate datasets, as the original datasets are quite large. These two datasets are:

1. df1_final: This dataset will be used to explore geographic location data.
2. df2: This dataset will focus on exploring orders, delivery performance, customer preferences, purchase patterns, and more.

## 1) First dataset

In [23]:
df1=pd.merge(geo_location,customers,left_on='geolocation_zip_code_prefix',right_on='customer_zip_code_prefix')
df1=pd.merge(df1,orders,on='customer_id')
df1=pd.merge(df1,order_items,on='order_id')
df1.head()


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,delivery_timeframe (days),order_item_id,product_id,price
0,1037,-23.545621,-46.639292,sao paulo,SP,05b005e82f3f2dcc41b5504f00484e50,3ef8c1513fc80fb905ccd8d57da5e9b8,1037,sao paulo,SP,e260cc3baa8f4aae3fab201ef60e1127,delivered,2018-08-17 10:59:04,2018-08-21 17:51:40,0.0,1,d7a023eecac412d29d49735e9e9b629f,25.0
1,1037,-23.545187,-46.637855,são paulo,SP,05b005e82f3f2dcc41b5504f00484e50,3ef8c1513fc80fb905ccd8d57da5e9b8,1037,sao paulo,SP,e260cc3baa8f4aae3fab201ef60e1127,delivered,2018-08-17 10:59:04,2018-08-21 17:51:40,0.0,1,d7a023eecac412d29d49735e9e9b629f,25.0
2,1037,-23.546705,-46.640336,são paulo,SP,05b005e82f3f2dcc41b5504f00484e50,3ef8c1513fc80fb905ccd8d57da5e9b8,1037,sao paulo,SP,e260cc3baa8f4aae3fab201ef60e1127,delivered,2018-08-17 10:59:04,2018-08-21 17:51:40,0.0,1,d7a023eecac412d29d49735e9e9b629f,25.0
3,1037,-23.543883,-46.638075,são paulo,SP,05b005e82f3f2dcc41b5504f00484e50,3ef8c1513fc80fb905ccd8d57da5e9b8,1037,sao paulo,SP,e260cc3baa8f4aae3fab201ef60e1127,delivered,2018-08-17 10:59:04,2018-08-21 17:51:40,0.0,1,d7a023eecac412d29d49735e9e9b629f,25.0
4,1037,-23.546157,-46.639885,sao paulo,SP,05b005e82f3f2dcc41b5504f00484e50,3ef8c1513fc80fb905ccd8d57da5e9b8,1037,sao paulo,SP,e260cc3baa8f4aae3fab201ef60e1127,delivered,2018-08-17 10:59:04,2018-08-21 17:51:40,0.0,1,d7a023eecac412d29d49735e9e9b629f,25.0


From the merged dataset, final dataset is filtered through the set of combined aggregations. 
The dataset is grouped by zip code to calculate median latitude and longitude (To find the central location for each zip code prefix), mode of the customer city (To identify most common city for a zip code prefix), total price, and customer count for each zip code. Additionally, a frequency column is added to indicate the number of entries per zip code

In [24]:
def mode(x):
    return x.mode()


df1_final=df1.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'median',
    'geolocation_lng': 'median',
    'customer_city': mode,
    'price': 'sum',
    'customer_id':'count'
})

df1_final['frequency']=df1.groupby('geolocation_zip_code_prefix').size()

df1_final

,geolocation_lat,geolocation_lng,customer_city,price,customer_id,frequency
geolocation_zip_code_prefix,,,,,,
1003,-23.548977,-46.635313,sao paulo,989.89,11,11
1004,-23.549550,-46.634771,sao paulo,2505.86,28,28
1005,-23.549763,-46.636100,sao paulo,6377.67,78,78
1006,-23.550317,-46.636225,sao paulo,5843.60,14,14
1007,-23.550349,-46.637342,sao paulo,7287.68,64,64
...,...,...,...,...,...,...
99960,-27.953797,-52.029641,charrua,625.00,5,5
99965,-28.179542,-52.035551,agua santa,983.40,12,12
99970,-28.343257,-51.875470,ciriaco,3664.00,16,16


In [25]:
df1_final.isnull().sum()

geolocation_lat    0
geolocation_lng    0
customer_city      0
price              0
customer_id        0
frequency          0
dtype: int64

In [26]:
#save dataframe to CSV file
df1.to_csv('df1.csv',index=False)
df1_final.to_csv('df1_final.csv',index=False)


## 2) Second Dataset

In [27]:
#join all datasets together
df2=pd.merge(orders,order_items,on='order_id')
df2=pd.merge(df2,products_merged,on='product_id')
df2=pd.merge(df2,order_payments,on='order_id')
df2=pd.merge(df2,order_reviews,on='order_id')


df2.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,delivery_timeframe (days),order_item_id,product_id,price,product_category_name_english,payment_type,review_id,review_score,review_comment_message_english,review_score_classification
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,voucher,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,voucher,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-18 14:44:43,9.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,b46f1e34512b0f4c74a72398b03ca788,4,They should package the product better. The bo...,Positive
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-07 18:30:01,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,dc90f19c2806f1abba9e72ad3c350073,5,"I only found it small for six cups, but it's a...",Positive


In [28]:
#keep relevant columns
cus_join=customers[['customer_id','customer_unique_id']]
df2=pd.merge(df2,cus_join,on='customer_id')
df2.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,delivery_timeframe (days),order_item_id,product_id,price,product_category_name_english,payment_type,review_id,review_score,review_comment_message_english,review_score_classification,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive,7c396fd4830fd04220f754e42b4e5bff
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,voucher,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive,7c396fd4830fd04220f754e42b4e5bff
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,voucher,a54f0611adc9ed256b57ede6b6eb5114,4,"I haven't tested the product yet, but it arriv...",Positive,7c396fd4830fd04220f754e42b4e5bff
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-18 14:44:43,9.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,b46f1e34512b0f4c74a72398b03ca788,4,They should package the product better. The bo...,Positive,3a51803cc0d012c3b5dc8b7528cb05f7
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-07 18:30:01,7.0,1,87285b34884572647811a353c7ac498a,29.99,housewares,credit_card,dc90f19c2806f1abba9e72ad3c350073,5,"I only found it small for six cups, but it's a...",Positive,ef0996a1a279c26e7ecbd737be23d235


In [29]:
#check for duplicated values
df2.duplicated().sum()


2428

In [30]:
df2.shape

(115609, 16)

In [31]:
#renaming columns to shorten them
df2=df2.rename(columns={'delivery_timeframe (days)':'delivery_timeframe',
                        'product_category_name_english':'product_category',
                        'review_comment_message_english':'review_comment',
                        'review_score_classification':'review_type'})
df2['review_comment']=df2['review_comment'].astype(str)


In [32]:
#check for null values
df2.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_delivered_customer_date    0
delivery_timeframe               0
order_item_id                    0
product_id                       0
price                            0
product_category                 0
payment_type                     0
review_id                        0
review_score                     0
review_comment                   0
review_type                      0
customer_unique_id               0
dtype: int64

In [33]:
#check final dataset shape
df2.shape

(115609, 16)

In [34]:
#save dataframe to CSV file
df2.to_csv('df2.csv',index=False)